# WeatherPy
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

In partnership with Anne Niemiec.

The starter code and pseudo code for this challenge was provided and can be reviewed in python-api-challenge\assignment_source_files

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key
if weather_api_key == "YOUR KEY HERE!":
    import sys
    sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
    from api_keys_NRL import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# Define function for calculating linear regressions
def regress_line(x,y):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x,y)
    y_regress = x * slope + intercept
    line_text = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    print(f"The r-value is: {rvalue ** 2}")
    return y_regress, line_text

## Determine user's preferred data source

In [3]:
# Determine if user would like to load pre-populated weather data or
get_current_wx = input("Would you like to analyze current weather conditions (Y/N)?")

# If no, load provided data
if get_current_wx == "N":
    print("The user opted to load pre-populated weather data")
    input_file = "input_data\cities.csv"
    df_wx_data = pd.read_csv(input_file)
elif get_current_wx == "Y":
    print("The user opted to analyze current weather conditions")
else:
    print("The user chose an invalid option.")
    raise SystemExit("Stop executing notebook") 

The user chose an invalid option.


SystemExit: Stop executing notebook

## Generate Cities List

In [ ]:
if get_current_wx == "Y":
    # List for holding lat_lngs and cities
    lat_lngs = []
    cities = []

    # Create a set of random lat and lng combinations
    lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
    lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
    lat_lngs = zip(lats, lngs)

    # Identify nearest city for each lat, lng combination
    for lat_lng in lat_lngs:
        city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
        
        # If the city is unique, then add it to a our cities list
        if city not in cities:
            cities.append(city)

    # Print the city count to confirm sufficient count
    print(len(cities))
else:
    print(len(df_wx_data))

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
if get_current_wx == "Y":
    # Setup and call weather API

    # Save config information.
    url = "http://api.openweathermap.org/data/2.5/weather?"
    units = "imperial"

    # Build partial query URL
    query_url = f"{url}appid={weather_api_key}&units={units}&q="

    # set up lists to hold reponse info
    city_list = []
    lat_list = []
    lon_list = []
    max_temp_list = []
    humidity_list = []
    cloudiness_list = []
    wndspd_list = []
    country_list = []
    date_list = []
    call_counter = 1
    call_set = 1

    # Collect weather data using json api calls
    print('Beginning Data Retrieval')     
    print('-----------------------------')

    # Loop through the list of cities and perform a request for data on each and populate data lists
    for city in cities:
        response = requests.get(query_url + city).json()
        #print(json.dumps(response, indent=4, sort_keys=True))

        try:
            city_list.append(response['name'])
            lat_list.append(response['coord']['lon'])
            lon_list.append(response['coord']['lon'])
            max_temp_list.append(response['main']['temp_max'])
            humidity_list.append(response['main']['humidity'])
            cloudiness_list.append(response['clouds']['all'])
            wndspd_list.append(response['wind']['speed'])
            country_list.append(response['sys']['country'])
            date_list.append(response['dt'])

            print(f'Processing Record {call_counter} of Set {call_set} | {city}')

        except KeyError:
            print("City not found. Skipping...")

        if call_counter > 49:
            call_counter = 1
            call_set += 1
            time.sleep(10)    
        else:
            call_counter += 1

    print('-----------------------------')
    print('Data Retrieval Complete')      
    print('-----------------------------')

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
if get_current_wx == "Y":
    df_wx_data = pd.DataFrame({
        "City": city_list,
        "Lat": lat_list,
        "Lng": lon_list,
        "Max Temp": max_temp_list,
        "Humidity": humidity_list,
        "Cloudiness": cloudiness_list,
        "Wind Speed": wndspd_list,
        "Country": country_list,
        "Date": date_list})
else:
    df_wx_data = df_wx_data.loc[:,["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
df_wx_data.to_csv(output_data_file)
df_wx_data

### Display summary statistics on the weather DataFrame

In [ ]:
df_wx_data.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Check for cities that have humidity over 100%.
df_invalid_humidity = df_wx_data.loc[df_wx_data["Humidity"] > 100, ["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
df_invalid_humidity

In [ ]:
# Remove invalid humidity data
df_wx_data_clean = df_wx_data.loc[df_wx_data["Humidity"] <= 100, ["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
df_wx_data_clean.describe()

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [ ]:
# Extract data collection date
pretty_date = []
for idx, row in df_wx_data_clean.iterrows():
    data_time = time.localtime(row["Date"])
    pretty_date.append(f"{data_time.tm_mon}/{data_time.tm_mday}/{data_time.tm_year}")

collection_date = str(set(pretty_date)).strip("{'}")
print(f"Data collected on {collection_date}")

## Latitude vs. Temperature Plot

In [ ]:
df_wx_data_clean.plot(
    "Lat","Max Temp",
    title=f"City Latitude vs Max Temperature ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.savefig("output_data/lat_temp.png",facecolor='w')
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
df_wx_data_clean.plot(
    "Lat","Humidity",
    title=f"City Latitude vs Humidity ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.savefig("output_data/lat_humidity.png",facecolor='w')
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
df_wx_data_clean.plot(
    "Lat","Cloudiness",
    title=f"City Latitude vs Cloudiness ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.savefig("output_data/lat_cloudiness.png",facecolor='w')
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
df_wx_data_clean.plot(
    "Lat","Wind Speed",
    title=f"City Latitude vs Wind Speed ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.savefig("output_data/lat_wndspd.png",facecolor='w')
plt.show()

## Linear Regression

Create separate figures for each hemisphere
* Northern hemisphere
* Southern hemisphere

In [ ]:
# Collect data by northern and southern hemispheres
df_wx_data_clean_N_hemi = df_wx_data_clean.loc[df_wx_data_clean["Lat"]>=0]
df_wx_data_clean_S_hemi = df_wx_data_clean.loc[df_wx_data_clean["Lat"]<0]
df_wx_data_clean_S_hemi

### Latitude vs. Temperature Plots with linear regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_N_hemi["Lat"],df_wx_data_clean_N_hemi["Max Temp"])

# Plot the data
df_wx_data_clean_N_hemi.plot(
    "Lat","Max Temp",
    title=f"Northern Hemisphere City Latitude vs Max Temperature ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_N_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(0,20),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

plt.savefig("output_data/north_lat_temp.png",facecolor='w')
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_S_hemi["Lat"],df_wx_data_clean_S_hemi["Max Temp"])

# Plot the data
df_wx_data_clean_S_hemi.plot(
    "Lat","Max Temp",
    title=f"Southern Hemisphere City Latitude vs Max Temperature ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_S_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(-30,40),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

plt.savefig("output_data/south_lat_temp.png",facecolor='w')
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_N_hemi["Lat"],df_wx_data_clean_N_hemi["Humidity"])

# Plot the data
df_wx_data_clean_N_hemi.plot(
    "Lat","Humidity",
    title=f"Northern Hemisphere City Latitude vs Humidity ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_N_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(45,10),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

plt.savefig("output_data/north_lat_humidity.png",facecolor='w')
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_S_hemi["Lat"],df_wx_data_clean_S_hemi["Humidity"])

# Plot the data
df_wx_data_clean_S_hemi.plot(
    "Lat","Humidity",
    title=f"Southern Hemisphere City Latitude vs Humidity ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_S_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(-50,20),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

plt.savefig("output_data/south_lat_humidity.png",facecolor='w')
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_N_hemi["Lat"],df_wx_data_clean_N_hemi["Cloudiness"])

# Plot the data
df_wx_data_clean_N_hemi.plot(
    "Lat","Cloudiness",
    title=f"Northern Hemisphere City Latitude vs Cloudiness ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_N_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(45,10),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

plt.savefig("output_data/north_lat_cloudiness.png",facecolor='w')
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_S_hemi["Lat"],df_wx_data_clean_S_hemi["Cloudiness"])

# Plot the data
df_wx_data_clean_S_hemi.plot(
    "Lat","Cloudiness",
    title=f"Southern Hemisphere City Latitude vs Cloudiness ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_S_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(-55,20),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

plt.savefig("output_data/south_lat_cloudiness.png",facecolor='w')
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_N_hemi["Lat"],df_wx_data_clean_N_hemi["Wind Speed"])

# Plot the data
df_wx_data_clean_N_hemi.plot(
    "Lat","Wind Speed",
    title=f"Northern Hemisphere City Latitude vs Wind Speed ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_N_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(30,27),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Wind Speed (MPH)")

plt.savefig("output_data/north_lat_wndspd.png",facecolor='w')
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Calculate Linear Regression
y_regress, line_text = regress_line(df_wx_data_clean_S_hemi["Lat"],df_wx_data_clean_S_hemi["Wind Speed"])

# Plot the data
df_wx_data_clean_S_hemi.plot(
    "Lat","Wind Speed",
    title=f"Southern Hemisphere City Latitude vs Wind Speed ({collection_date})",
    kind='scatter',
    grid=True,
    style="o",
    facecolors="blue",
    edgecolors="black",
    alpha = 0.8,
    s=30
    )

plt.plot(df_wx_data_clean_S_hemi["Lat"],y_regress,"r-")
plt.annotate(line_text,(-30,32),fontsize=15,color="red")

plt.xlabel("Latitude")
plt.ylabel("Wind Speed (MPH)")

plt.savefig("output_data/south_lat_wndspd.png",facecolor='w')
plt.show()